<h1>Sentiment Analysis of IMDb Movie Reviews using LSTM and Keras:</h1> 
    <p> - A Comprehensive Guide by Aminu Mohammed Twumasi</p>
    <hr>

<h3>This guide provides a step-by-step approach to sentiment analysis of IMDb movie reviews using LSTM and Keras.</h3> The process involves:

<ol>
    <li><h4>Data Acquisition:</h4> Downloading the 50K IMDb Movie Review dataset</li>
    
  <li><h4>Data Preprocessing:</h4> Cleaning and preparing the dataset for analysis</li>
    
  <li><h4>Sentiment Encoding:</h4> Converting sentiment labels into numerical representations</li>
    
  <li><h4>Data Splitting:</h4> Dividing the dataset into training and testing sets</li>
    
  <li><h4>Text Tokenization:</h4> Transforming text reviews into numerical sequences</li>
    
  <li><h4>Model Framework/Architecture/Building:</h4> Constructing an LSTM-based neural network architecture</li>
    
  <li><h4>Model Training:</h4> Training the model on the training set</li>
    
  <li><h4>Model Evaluation:</h4> Assessing the model's performance on the testing set</li>
</ol>

In [1]:
# Import necessary libraries
import pandas as pd  # For loading the dataset
import numpy as np  # For mathematical operations
from nltk.corpus import stopwords  # For accessing a collection of stopwords
from sklearn.model_selection import train_test_split  # For splitting the dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # For encoding text to integers
from tensorflow.keras.preprocessing.sequence import pad_sequences  # For padding or truncating sequences
from tensorflow.keras.models import Sequential  # For building the model
from tensorflow.keras.layers import Embedding, LSTM, Dense  # Defining the model architecture
from tensorflow.keras.callbacks import ModelCheckpoint  # For saving the model
from tensorflow.keras.models import load_model  # For loading a saved model
import re  # For regular expression operations

2023-11-08 21:41:49.854173: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


<h3>Data Acquisition</h3>

In [2]:
# Load the dataset with Pandas
data = pd.read_csv('/home/sir/Documents/IMDB_Dataset.csv')

# Preview the dataset
print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [3]:
# Stopwords are commonly used words in a sentence that are often ignored by search engines, as they do not add much meaning to the search query. Examples of stopwords include "the", "a", "an", "of", etc.
# Declare the English stopwords for later use

english_stops = set(stopwords.words('english'))

<h3>Dataset Loading and Cleaning</h3>
<hr>
The initial dataset contains unclean reviews with HTML tags, numbers, uppercase letters, and punctuation marks. This can negatively impact training. Therefore, within the "load_dataset()" function, I load the dataset using pandas and perform preprocessing on the reviews. This involves removing HTML tags, non-alphabetic characters (punctuation and numbers), stop words, and converting all reviews to lowercase.

<h3>Sentiment Encoding</h3>
<hr>
Additionally, within the same function, I encode the sentiments as integers. Negative sentiments are encoded as 0, while positive sentiments are encoded as 1.

In [4]:
def load_dataset():
    """
    Loads the IMDb dataset and preprocesses the reviews and sentiment labels.

    Returns:
        x_data: The reviews, preprocessed and encoded as integers.
        y_data: The sentiment labels, encoded as 0 or 1.
    """

    # Read the IMDb dataset from a CSV file
    df = pd.read_csv('/home/sir/Documents/IMDB_Dataset.csv')

    # Extract the reviews and sentiment labels
    x_data = df['review']
    y_data = df['sentiment']

    # Preprocess the reviews
    # Remove HTML tags
    x_data = x_data.replace({'<.*?>': ''}, regex=True)

    # Remove non-alphabetical characters
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex=True)

    # Remove stop words (e.g., "the", "a", "an")
    english_stops = set(stopwords.words('english'))  # Define the list of English stopwords
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])

    # Convert all words to lowercase
    x_data = x_data.apply(lambda review: [w.lower() for w in review])

    # Encode the sentiment labels
    # Replace 'positive' with 1
    y_data = y_data.replace('positive', 1)

    # Replace 'negative' with 0
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

# Load the preprocessed reviews and sentiment labels
x_data, y_data = load_dataset()

# Print the reviews
print('Reviews:')
print(x_data)
print('\n')

# Print the sentiment labels
print('Sentiment:')
print(y_data)

Reviews:
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object


Sentiment:
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


In [5]:
# Print the reviews
print('Reviews:')
print(x_data)
print('\n')

# Print the sentiment labels
print('Sentiment:')
print(y_data)

Reviews:
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object


Sentiment:
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


<h3>Dataset Splitting</h3>
<hr>
To ensure accurate predictions, I have chosen to split the data into a training set comprising 80% of the data and a testing set comprising 20% of the data. This split is achieved using the "train_test_split" method from Scikit-Learn. The method automatically shuffles the dataset, which is necessary because the original dataset lists positive reviews first, followed by negative reviews. By shuffling the data, it is distributed evenly in the model, enhancing prediction accuracy.

In [6]:
# Split the preprocessed data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2)

# Print the training reviews
print('Train Set:')
print(x_train)
print('\n')

# Print the testing reviews
print(x_test)
print('\n')

# Print the training sentiment labels
print('Train Set')
print(y_train)
print('\n')

# Print the testing sentiment labels
print('Test Set')
print(y_test)

Train Set:
2163     [i, belgium, therefore, english, writing, rath...
13817    [as, half, big, fans, trash, horror, resist, g...
21006    [gritty, drama, emotionally, powerful, blah, t...
38895    [tempest, somewhat, self, indulgent, uneven, d...
48257    [if, like, horror, movies, lots, blood, gore, ...
                               ...                        
26542    [it, really, shame, imdb, let, give, negative,...
45826    [well, said, horror, comedy, features, neither...
40777    [in, lassie, come, home, national, velvet, the...
47972    [there, things, i, never, understand, such, mo...
28704    [if, like, occasionally, enjoy, watching, terr...
Name: review, Length: 40000, dtype: object


29699    [i, give, two, lot, music, otherwise, would, o...
24396    [did, movie, makers, even, preview, released, ...
22755    [in, group, raf, officers, including, eric, wi...
38911    [watching, movie, waste, time, i, tempted, lea...
14107    [if, vampire, tales, cup, blood, goth, fest, b...


<h3>Function to Get Maximum Review Length</h3>
<hr>
To determine the maximum review length, I have created a function that calculates the mean of all the reviews' length using the "numpy.mean" method. This provides a good estimate of the maximum review length.

In [7]:
def get_max_length():
    """
    Determines the maximum length of any review in the training set.

    Returns:
        int: The maximum length of any review.
    """

    review_length = []  # Initialize an empty list to store review lengths

    # Iterate through all the reviews in the training set
    for review in x_train:
        # Append the length of the current review to the `review_length` list
        review_length.append(len(review))

    # Calculate the average review length
    average_review_length = np.mean(review_length)

    # Round the average review length up to the nearest integer
    max_length = int(np.ceil(average_review_length))

    return max_length

<h3>Text/Review Tokenization and Padding/Truncation</h3>
<hr>
To prepare the reviews for input into a neural network, we need to encode them into numeric form. I utilize the "tensorflow.keras.preprocessing.text.Tokenizer" class to accomplish this. The "fit_on_texts" method is used to automatically index each unique word based on the "x_train" data.

Both "x_train" and "x_test" are then converted into sequences of integers using the "texts_to_sequences" method.

Since the reviews vary in length, we need to ensure that they are of the same length for the neural network. This is achieved by adding padding (0) or truncating words. The "tensorflow.keras.preprocessing.sequence.pad_sequences" method is employed for this purpose.

You can choose to either pad or truncate the words at the end of a sentence (post) or at the beginning of a sentence (pre).

In [8]:
# ENCODE REVIEW
# Initialize a Tokenizer object from the tensorflow.keras.preprocessing.text module. Set the lower parameter to False since the reviews have already been converted to lowercase during data loading.
# Create a Tokenizer object:
token = Tokenizer(lower=False) 

# Use the fit_on_texts() method to fit the Tokenizer on the training reviews. This involves creating a vocabulary of unique words and assigning them numerical indices.
# Fit the Tokenizer on the training reviews:
token.fit_on_texts(x_train)

# Convert the training reviews from text sequences to sequences of integers using the texts_to_sequences() method of the Tokenizer. This replaces each word with its corresponding numerical index from the vocabulary.
# Encode the training reviews:
x_train = token.texts_to_sequences(x_train)

# Similarly, convert the testing reviews from text sequences to sequences of integers using the texts_to_sequences() method. This ensures that the training and testing reviews are encoded consistently.
# Encode the testing reviews:
x_test = token.texts_to_sequences(x_test)

# Call the get_max_length() function to determine the maximum length of any review in the training set. This information is crucial for padding or truncating reviews to a uniform length.
# Review Padding and Truncating
# Determine the maximum review length:
max_length = get_max_length()

# Use the pad_sequences() function to pad or truncate the training reviews to the max_length. The padding parameter is set to 'post', indicating that padding should be done at the end of the sequences. The truncating parameter is also set to 'post', indicating that longer sequences should be truncated from the end if necessary.
# Pad or truncate the training reviews:
x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')

# Apply the same padding and truncating process to the testing reviews using the pad_sequences() function. This ensures that both training and testing reviews have the same length.
# Pad or truncate the testing reviews:
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

# Determine the total number of unique words in the vocabulary by calculating the length of the token.word_index dictionary. Add 1 to account for the padding index (0).
# Vocabulary Size Calculation
# Calculate the total number of unique words:
total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[    1 10988  1490 ...     0     0     0]
 [  111   207    99 ...     0     0     0]
 [ 2379   345  2175 ...     0     0     0]
 ...
 [   50 11492   123 ... 12359  9955 11238]
 [   49    89     1 ...     0     0     0]
 [   56     6  1845 ...     0     0     0]] 

Encoded X Test
 [[    1   105    36 ...   973   505  1646]
 [ 1441     3  1146 ...     0     0     0]
 [   50   443 15090 ...    54     0     0]
 ...
 [   50 45776  1948 ...     1  2299    43]
 [25550   602   129 ...     0     0     0]
 [    8     4  2356 ...     0     0     0]] 

Maximum review length:  130


<h3>Model Architecture</h3>
<hr>
The model architecture consists of three main layers: Embedding, LSTM, and Dense.

The Embedding layer creates word vectors for each word in the "word_index" by analyzing the other words around them. This helps group together words that are related or have similar meanings.

The LSTM layer is responsible for making decisions on whether to keep or throw away data by considering the current input, previous output, and previous memory. It comprises several important components, including the Forget Gate, Input Gate, Cell State, and Output Gate.

The Dense layer computes the input with the weight matrix and bias (optional) using an activation function. In this work, I use the Sigmoid activation function since the output is only 0 or 1.

The optimizer used is Adam, and the loss function is Binary Crossentropy since the output is a binary number (0 or 1).

In [9]:
EMBED_DIM = 32  # Define the embedding dimension (number of units in the embedding layer)
LSTM_OUT = 64  # Define the number of units in the LSTM layer

# Initialize a Sequential model
model = Sequential()

# Add an Embedding layer to convert words into vectors
model.add(Embedding(total_words, EMBED_DIM, input_length=max_length))
    # total_words: The size of the vocabulary (number of unique words)
    # EMBED_DIM: The dimensionality of the embedding vectors
    # input_length: The maximum length of any review (after padding or truncating)

# Add an LSTM layer to learn sequential patterns in the reviews
model.add(LSTM(LSTM_OUT))
    # LSTM_OUT: The number of units in the LSTM layer

# Add a Dense layer to produce a binary probability for sentiment prediction
model.add(Dense(1, activation='sigmoid'))
    # 1: The output dimension (1 unit for binary classification)
    # activation='sigmoid': Use the sigmoid activation function for binary probability output

# Compile the model using the Adam optimizer, binary cross-entropy loss, and accuracy metric
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    # optimizer='adam': Use the Adam optimizer for weight updates
    # loss='binary_crossentropy': Use binary cross-entropy loss for binary classification
    # metrics=['accuracy']: Evaluate the model's accuracy during training and evaluation

# Print a summary of the model architecture
print(model.summary())

2023-11-08 21:48:33.408769: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 130, 32)           2954304   
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 2,979,201
Trainable params: 2,979,201
Non-trainable params: 0
_________________________________________________________________
None


2023-11-08 21:48:33.855337: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-11-08 21:48:33.857197: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-11-08 21:48:33.858838: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

<h3>Training Process</h3>
<hr>
To train the model, we simply need to fit our "x_train" (input) and "y_train" (output/label) data. In this training process, I employ a mini-batch learning method with a batch size of 128 and 5 epochs.

Additionally, I have included a callback named "checkpoint" to save the model locally after each epoch if its accuracy has improved compared to the previous epoch.

In [10]:
# ARCHITECTURE
EMBED_DIM = 32  # Define the embedding dimension (number of units in the embedding layer)
LSTM_OUT = 64  # Define the number of units in the LSTM layer

# Initialize a Sequential model
model = Sequential()

# Add an Embedding layer to convert words into vectors
model.add(Embedding(total_words, EMBED_DIM, input_length=max_length))
    # total_words: The size of the vocabulary (number of unique words)
    # EMBED_DIM: The dimensionality of the embedding vectors
    # input_length: The maximum length of any review (after padding or truncating)

# Add an LSTM layer to learn sequential patterns in the reviews
model.add(LSTM(LSTM_OUT))
    # LSTM_OUT: The number of units in the LSTM layer

# Add a Dense layer to produce a binary probability for sentiment prediction
model.add(Dense(1, activation='sigmoid'))
    # 1: The output dimension (1 unit for binary classification)
    # activation='sigmoid': Use the sigmoid activation function for binary probability output

# Compile the model using the Adam optimizer, binary cross-entropy loss, and accuracy metric
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    # optimizer='adam': Use the Adam optimizer for weight updates
    # loss='binary_crossentropy': Use binary cross-entropy loss for binary classification
    # metrics=['accuracy']: Evaluate the model's accuracy during training and evaluation

# Print a summary of the model architecture
print(model.summary())

In [13]:
# Train the model using the training data
model.fit(x_train, y_train, batch_size=128, epochs=5, callbacks=[checkpoint])

# Explain the code:

# model.fit(x_train, y_train):
# This line of code starts the training process of the model. It takes three arguments:
# x_train: The encoded training reviews (sequences of integers)
# y_train: The sentiment labels for the training reviews (0 for negative, 1 for positive)
# batch_size: The number of training examples to process in one batch during training (128 in this case)
# epochs: The number of times to iterate through the entire training dataset (5 in this case)

# callbacks=[checkpoint]:
# This part specifies a callback to use during training. A callback is a function that can be called at specific stages of the training process, such as the end of an epoch.
# The checkpoint callback is used to save the model weights at regular intervals. This allows you to save the best model during training and use it for inference later.


Epoch 1/5
313/313 [==============================] - ETA: 0s - loss: 0.0513 - accuracy: 0.9862
Epoch 1: accuracy improved from 0.98480 to 0.98620, saving model to models/LSTM.h5
313/313 [==============================] - 101s 324ms/step - loss: 0.0513 - accuracy: 0.9862
Epoch 2/5
313/313 [==============================] - ETA: 0s - loss: 0.0440 - accuracy: 0.9887
Epoch 2: accuracy improved from 0.98620 to 0.98870, saving model to models/LSTM.h5
313/313 [==============================] - 98s 313ms/step - loss: 0.0440 - accuracy: 0.9887
Epoch 3/5
313/313 [==============================] - ETA: 0s - loss: 0.0315 - accuracy: 0.9929
Epoch 3: accuracy improved from 0.98870 to 0.99292, saving model to models/LSTM.h5
313/313 [==============================] - 75s 240ms/step - loss: 0.0315 - accuracy: 0.9929
Epoch 4/5
313/313 [==============================] - ETA: 0s - loss: 0.0297 - accuracy: 0.9931
Epoch 4: accuracy improved from 0.99292 to 0.99308, saving model to models/LSTM.h5
313/313 [==

<h3>Testing and Evaluation</h3>
<hr>
To evaluate the model, we predict the sentiment using the "x_test" data and compare the predictions with the "y_test" (expected output) data. We then calculate the accuracy of the model by dividing the number of correct predictions by the total data. 

The resulting accuracy of the model is 86.63%.

In [16]:
# Make predictions on the testing data
y_pred = model.predict(x_test, batch_size=128)

# Convert the predicted probabilities to class labels (0 or 1)
y_pred_classes = np.argmax(y_pred, axis=1)

# Initialize a counter for correct predictions
true = 0

# Iterate through the testing data and compare predictions to actual labels
for i, y in enumerate(y_test):
    if y == y_pred_classes[i]:
        true += 1

# Calculate the number of correct and incorrect predictions
correct_predictions = true
wrong_predictions = len(y_pred_classes) - true

# Calculate the accuracy
accuracy = correct_predictions / len(y_pred_classes) * 100

# Print the number of correct and incorrect predictions
print('Correct Prediction: {}'.format(correct_predictions))
print('Wrong Prediction: {}'.format(wrong_predictions))

# Print the accuracy as a percentage
print('Accuracy: {:.2f}%'.format(accuracy))


2023-11-08 22:06:27.088356: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-11-08 22:06:27.090431: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-11-08 22:06:27.092142: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

79/79 [==============================] - 9s 110ms/step
Correct Prediction: 5024
Wrong Prediction: 4976
Accuracy: 50.239999999999995


<h3> To LOAD THE SAVED MODEL and use it for predicting the sentiment of a movie review statement, you can follow these steps:</h3>
<hr>
1. Use the loaded model to predict the sentiment of the preprocessed movie review statement. The model will output a probability or a binary value indicating the sentiment (positive or negative).

2. Interpret the prediction result based on the output of the model. For example, if the model outputs a probability, you can set a threshold (e.g., 0.5) to classify it as either positive or negative. If the model outputs a binary value, you can directly interpret it as the sentiment.

In [17]:
loaded_model = load_model('models/LSTM.h5')

2023-11-08 22:07:05.194165: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-11-08 22:07:05.196002: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-11-08 22:07:05.197547: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Receives a review as an input to be predicted

In [18]:
review = str(input('Movie Review: '))

Movie Review: Nothing was typical about this. Everything was beautifully done in this movie, the story, the flow, the scenario, everything. I highly recommend it for mystery lovers, for anyone who wants to watch a good movie!


In [19]:
# Make predictions on the testing data
y_pred = model.predict(x_test, batch_size=128)

# Convert the predicted probabilities to class labels (0 or 1)
y_pred_classes = np.argmax(y_pred, axis=1)

# Initialize a counter for correct predictions
true = 0

# Iterate through the testing data and compare predictions to actual labels
for i, y in enumerate(y_test):
    if y == y_pred_classes[i]:
        true += 1

# Calculate the number of correct and incorrect predictions
correct_predictions = true
wrong_predictions = len(y_pred_classes) - true

# Calculate the accuracy
accuracy = correct_predictions / len(y_pred_classes) * 100

# Print the number of correct and incorrect predictions
print('Correct Prediction: {}'.format(correct_predictions))
print('Wrong Prediction: {}'.format(wrong_predictions))

# Print the accuracy as a percentage
print('Accuracy: {:.2f}%'.format(accuracy))


Cleaned:  Nothing was typical about this Everything was beautifully done in this movie the story the flow the scenario everything I highly recommend it for mystery lovers for anyone who wants to watch a good movie
Filtered:  ['nothing typical everything beautifully done movie story flow scenario everything i highly recommend mystery lovers anyone wants watch good movie']


To predict the sentiment of a movie review statement, we first need to tokenize and encode the words in the statement. In this case, we can use the tokenizer that was previously declared and fitted on the training data. This ensures that the words in the statement are encoded based on the word index that is known by the model.

By tokenizing and encoding the words using the existing tokenizer, we can prepare the statement for input into the model and obtain the predicted sentiment.

In [20]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

[[  76  702  173 1182  127    3   13 2685 2652  173    1  455  279  683
  1778  151  395   33    9    3    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]]


This is the result of the prediction which shows the confidence score of the review statement.

In [21]:
result = loaded_model.predict(tokenize_words)
print(result)

1/1 [==============================] - 1s 752ms/step
[[0.9971558]]


2023-11-08 22:09:06.275102: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-11-08 22:09:06.277278: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-11-08 22:09:06.279062: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

<h3>Threshold for Sentiment Prediction</h3>
<hr>
To interpret the output of the model, you can use a threshold value to determine whether the sentiment is positive or negative. A threshold value of 0.7 is a reasonable choice to classify the sentiment as positive or negative.

If the confidence score output by the model is close to 0, then the statement is negative. On the other hand, if the confidence score is close to 1, then the statement is positive. If the confidence score is equal to or greater than 0.7, it is classified as positive, and if it is less than 0.7, it is classified as negative.

In [22]:
if result >= 0.7:
    print('positive')
else:
    print('negative')

positive
